# The future home of an awesome ML project.
Please keep things commented and use lots of Markdown cells to separate sections or steps of the process, or to provide additional explanation.

In [ ]:
#!pip install <package_name>
#import <package_name>